Author: Micheal C. Chen

Email: muchuchen03@gmail.com

# Task 1
In this task, we will use variational matrix product state (MPS) method to solve a 1D **Cluster-Ising** chain with open boundary condition (OBC):
$$
H = \sum_j \left[-\left(X_j + Z_j Z_{j+1}\right) + g\left(X_jZ_{j+1}Z_{j+2}+Z_jZ_{j+1}X_{j+2}\right)\right]
$$
where $X,Y,Z$ denote Pauli $X,Y,Z$ matrices.
* For single-body terms, $j=1,\ldots, N$
* For two-body terms, $(j, j +1) = (1, 2), (2, 3),\ldots, (N-1, N)$
* For three-body terms, $(j, j+1, j+2) = (1, 2, 3), (2, 3, 4), \ldots, (N-2, N-1, N)$

Use the parameters:  $N=10, g=0.248, D_s = 4 $ or $6$

## Tasks
1. Write the matrix product operator (MPO) form of the Hamiltonian and draw the finite automata.
2. Calculate ground state energy $E$ per site, and magnetization per site $\langle Z_i \rangle$ and  $\langle X_i \rangle$.
3. Use 2-site variational MPS method and compare with 1-site variational MPS method and exact diagonalization.

In [1]:
import numpy as np
import numpy.linalg as LA
import Sub180221 as sub
import scipy.sparse.linalg as LAs
import math, copy

Firstly, we define Pauli operators and write Hamiltonian in MPO form.

In [2]:
def GenPauliOp():
    """
    Return four Pauli matrices I, X, Y, Z.
    """
    I = np.array([[1, 0], [0, 1]], dtype = complex)
    X = np.array([[0, 1], [1, 0]], dtype = complex)
    Y = np.array([[0, -1j],[1j, 0]], dtype = complex)
    Z = np.array([[1, 0], [0, -1]], dtype = complex)
    return I, X, Y, Z

The lattice Hamiltonian can be represented by the finite automata
<p align="center">
  <img src="Finite-Automata.png" alt="Finite Automata of the MPO" width="360">
  
</p>

And we find $D_{Mpo}=6$, then we construct translation-invariant MPO form Hamiltonian. The matrix form:
$$
M = 
\begin{pmatrix}
 I & X & Z & 0 & 0 & -X \\
 0 & 0 & 0 & Z & 0 & 0  \\
 0 & 0 & 0 & 0 & Z & -Z \\
 0 & 0 & 0 & 0 & 0 & gZ \\
 0 & 0 & 0 & 0 & 0 & gX \\
 0 & 0 & 0 & 0 & 0 & I
\end{pmatrix}
$$

In [3]:
def GenMpo_ClusterIsing_Obc(Dmpo, g, Dp = 2):
    """
    Args:
        Dmpo: Virtual bond dimension of Matrix Product Operator (MPO).
        g: Cluster interaction strength.
        Dp: Physical bond dimension, Dp=2
    Return:
        Mpo: Result Matrix Product Operator of Cluster-Isnig model.
    """
    Mpo = np.zeros((Dmpo, Dp, Dmpo, Dp), dtype = complex)
    I, X, _, Z = GenPauliOp()
    Mpo[0, :, 0, :] = I
    Mpo[0, :, 1, :] = X
    Mpo[0, :, 2, :] = Z
    Mpo[0, :, 5, :] = -X
    Mpo[1, :, 3, :] = Z
    Mpo[2, :, 4, :] = Z
    Mpo[2, :, 5, :] = -Z
    Mpo[3, :, 5, :] = g * Z
    Mpo[4, :, 5, :] = g * X
    Mpo[5, :, 5, :] = I
    
    return Mpo

We then randomly initialize a MPS ansatz with right-canonical form, and corresponding environment.

In [4]:
def InitMPS(Ns, Dp, Ds, Mpo):
    """
    Args:
        Ns: Number of sites.
        Dp: Physical bond dimension.
        Ds: Max virtual bond dimension.
        Mpo: Matrix product operator.
    Returns:
        Mps: A random MPS with right canonical form.
        EnvLeft: Left environment.
        EnvRight: Right environment.
    """
    Dmpo = np.shape(Mpo)[0]
    Mps = [None] * Ns
    EnvLeft = [None] * Ns
    EnvRight = [None] * Ns
    for site in range(Ns):
        left_bond_dim = min(Dp**site, Dp**(Ns-site), Ds)
        right_bond_dim = min(Dp**(site+1), Dp**(Ns-site-1), Ds)
        Mps[site] = np.random.rand(left_bond_dim, Dp, right_bond_dim) + 1j * np.random.rand(left_bond_dim, Dp, right_bond_dim)

    Ur = np.eye(np.shape(Mps[-1])[-1])
    for site in range(Ns-1, -1, -1):
        Ur, Mps[site] = sub.Mps_LQP(Mps[site], Ur)

    EnvLeft[0] = np.zeros((1, Dmpo, 1), dtype = complex)
    EnvLeft[0][0, 0, 0] = 1.0

    EnvRight[-1] = np.zeros((1, Dmpo, 1), dtype = complex)
    EnvRight[-1][0, -1, 0] = 1.0

    for site in range(Ns-1, 0, -1):
        EnvRight[site-1] = sub.NCon([EnvRight[site], Mps[site], Mpo, np.conj(Mps[site])],
                                    [[1, 3, 5], [-1, 2, 1], [-2, 2, 3, 4], [-3, 4, 5]])
    
    return Mps, EnvLeft, EnvRight

Before perform variational MPS method, we can use 1-site update and 2-site update. Both of the update method are equivalent of eigen equation $\hat{H}x = Ex$

In [5]:
def OptMpsSite(Mpo, EnvLeft, EnvRight, Tensor):
    """
    Args:
        Mpo: Matrix product operator of Cluster-Ising model.
        EnvLeft: Left Environment.
        EnvRight: Right Environment.
        Tensor: The tensor to be updated.
    Return:
        Tensor: Updated local tensor.
        eig: eigenvalue.
    """
    # Optimazation equation is equivalent to Ax=λx
    DT = np.shape(Tensor)
    A = sub.NCon([EnvLeft, Mpo, EnvRight], [[-1, 1, -4], [1, -5, 2, -2], [-6, 2, -3]])
    A = sub.Group(A, [[0, 1, 2], [3, 4, 5]])
    Eig, V = LAs.eigsh(A, k=1, which = 'SA')
    Tensor = np.reshape(V, DT)
    eig = float(Eig[0].real)
    return Tensor, eig

def OptMpsPair(Mpo, EnvLeft, EnvRight, TensorLeft, TensorRight):
    """
    Args:
        Mpo: Matrix product operator of Cluster-Ising model.
        EnvLeft: Left Environment.
        EnvRight: Right Environment.
        TensorLeft: Left tensor of the tensor pair to be updated.
        TensorRight: Right tensor of the tensor pair to be updated.
    Returns:
        TensorPair: Updated tensor pair (A rank-4 tensor)
        eig: eigenvalue.
    """
    DTL = np.shape(TensorLeft)
    DTR = np.shape(TensorRight)
    A = sub.NCon([EnvLeft, Mpo, Mpo, EnvRight], [[-1, 1, -5], [1, -6, 3, -2], [3, -7, 2, -3], [-8, 2, -4]])
    A = sub.Group(A, [[0, 1, 2, 3], [4, 5, 6, 7]])
    Eig, V = LAs.eigsh(A, k=1, which = 'SA')
    TensorPair = np.reshape(V, (DTL[0], DTL[1], DTR[1], DTR[-1]))
    eig = float(Eig[0].real)
    return TensorPair, eig

Then we perform variational MPS algorithm.

In [6]:
def OptMps(Mps, Mpo, EnvLeft, EnvRight, Ds, Method = 0):
    """
    Args:
        Mps: A randomly initialized MPS ansatz with right canonical form.
        Mpo: Matrix Product Operator of Cluster-Isnig model.
        EnvLeft: Corresponding left environment.
        EnvRight: Corresponding right environment.
        Ds: Max virtual bond dimension.
        Method: If Method == 0, we use 1-site update method. If Method == 1, we use 2-site update method.
    Returns:
        Mps: Updated MPS.
        Eng1: Ground state energy per site.
    """
    Ns = len(Mps)
    Eng0 = np.zeros(Ns)
    Eng1 = np.zeros(Ns)
    for sweep in range(100):
        for site in range(Ns-1):
            if Method == 0:
                Mps[site], Eng1[site] = OptMpsSite(Mpo, EnvLeft[site], EnvRight[site], Mps[site])
                Mps[site], U = sub.Mps_QR0P(Mps[site])
                EnvLeft[site+1] = sub.NCon([EnvLeft[site], np.conj(Mps[site]), Mpo, Mps[site]], [[1, 3, 5], [1, 2, -1], [3, 4, -2, 2], [5, 4, -3]])
                Mps[site+1] = np.tensordot(U, Mps[site+1], (1, 0))

            if Method == 1:
                TensorPair, Eng1[site] = OptMpsPair(Mpo, EnvLeft[site], EnvRight[site+1], Mps[site], Mps[site+1])
                Dl, Dpl, Dpr, Dr = np.shape(TensorPair)
                Mat = sub.Group(TensorPair, [[0, 1], [2, 3]])
                U, S, V = LA.svd(Mat, full_matrices = False)
                Dc = min(np.shape(U)[-1], Ds)
                U = U[:, :Dc]; S = S[:Dc]; Vh = V[:Dc, :]

                Mps[site] = np.reshape(U, (Dl, Dpl, Dc))
                Vh = np.reshape(Vh, (Dc, Dpr, Dr)); Smat = np.diag(S)
                Mps[site+1] = sub.NCon([Smat, Vh], [[-1, 1], [1, -2, -3]])

                EnvLeft[site+1] = sub.NCon([EnvLeft[site], np.conj(Mps[site]), Mpo, Mps[site]], [[1, 3, 5], [1, 2, -1], [3, 4, -2, 2], [5, 4, -3]])

        for site in range(Ns-1, 0, -1):
            if Method == 0:
                Mps[site], Eng1[site] = OptMpsSite(Mpo, EnvLeft[site], EnvRight[site], Mps[site])
                U, Mps[site] = sub.Mps_LQ0P(Mps[site])
                EnvRight[site-1] = sub.NCon([EnvRight[site], Mps[site], Mpo, np.conj(Mps[site])], [[1, 3, 5], [-1, 2, 1], [-2, 2, 3, 4], [-3, 4, 5]])
                Mps[site-1] = np.tensordot(Mps[site-1], U, (2, 0))

            if Method == 1:
                TensorPair, Eng1[site] = OptMpsPair(Mpo, EnvLeft[site-1], EnvRight[site], Mps[site-1], Mps[site])
                Dl, Dpl, Dpr, Dr = np.shape(TensorPair)
                Mat = sub.Group(TensorPair, [[0, 1], [2, 3]])
                U, S, V = LA.svd(Mat, full_matrices = False)
                Dc = min(np.shape(U)[-1], Ds)
                U = U[:, :Dc]; S = S[:Dc]; Vh = V[:Dc, :]

                Mps[site] = np.reshape(Vh, (Dc, Dpr, Dr))
                U = np.reshape(U, (Dl, Dpl, Dc)); Smat = np.diag(S)
                Mps[site-1] = sub.NCon([U, Smat], [[-1, -2, 1], [1, -3]])

                EnvRight[site-1] = sub.NCon([EnvRight[site], Mps[site], Mpo, np.conj(Mps[site])], [[1, 3, 5], [-1, 2, 1], [-2, 2, 3, 4], [-3, 4, 5]])
        if np.max(np.abs(Eng0-Eng1)) < 1.0e-7:
            break
        Eng0 = copy.copy(Eng1)


    return Mps, Eng1

After we get updated MPS, we can immediately calculate physical observables such as $\langle X_i \rangle$ and $\langle Z_i \rangle$.

In [7]:
def CalSigmaX_MPS(Mps, site):
    """
    Use MPS contraction method to compute X_i.
    """
    _, X, _, _ = GenPauliOp()
    Ns = len(Mps)
    EnvLeft = [None]* Ns
    EnvRight = [None] * Ns
    Dl0 = Mps[0].shape[0]
    EnvLeft[0] = np.eye(Dl0)
    DrN = Mps[-1].shape[2]
    EnvRight[-1] = np.eye(DrN)
    for i in range(site):
        EnvLeft[i+1] = sub.NCon([EnvLeft[i], np.conj(Mps[i]), Mps[i]], [[1, 3], [1, 2, -1], [3, 2, -2]])
    for i in range(Ns-1, site, -1):
        EnvRight[i-1] = sub.NCon([EnvRight[i], Mps[i], np.conj(Mps[i])], [[1, 3], [-1, 2, 1], [-2, 2, 3]])

    Val = sub.NCon([EnvLeft[site], EnvRight[site], Mps[site], np.conj(Mps[site]), X], [[1, 3], [4, 2], [3, 5, 4], [1, 6, 2], [5, 6]])
    return complex(Val)

def CalSigmaZ_MPS(Mps, site):
    """
    Use MPS contraction method to compute Z_i.
    """
    _, _, _, Z = GenPauliOp()
    Ns = len(Mps)
    EnvLeft = [None]* Ns
    EnvRight = [None] * Ns
    Dl0 = Mps[0].shape[0]
    EnvLeft[0] = np.eye(Dl0)
    DrN = Mps[-1].shape[2]
    EnvRight[-1] = np.eye(DrN)
    for i in range(site):
        EnvLeft[i+1] = sub.NCon([EnvLeft[i], np.conj(Mps[i]), Mps[i]], [[1, 3], [1, 2, -1], [3, 2, -2]])
    for i in range(Ns-1, site, -1):
        EnvRight[i-1] = sub.NCon([EnvRight[i], Mps[i], np.conj(Mps[i])], [[1, 3], [-1, 2, 1], [-2, 2, 3]])

    Val = sub.NCon([EnvLeft[site], EnvRight[site], Mps[site], np.conj(Mps[site]), Z], [[1, 3], [4, 2], [3, 5, 4], [1, 6, 2], [5, 6]])
    return complex(Val)

In [8]:
Dp = 2;Dmpo = 6;g = 0.428; Ns = 10 # Hamiltonian parameters
Mpo = GenMpo_ClusterIsing_Obc(Dmpo, g)
Ds_list = [4, 6]
Method_list = [0, 1]
for Ds in Ds_list:
    for Method in Method_list:
        print(f"(Ns, Ds, g)={Ns, Ds, g}")
        print(f"{Method+1}-site Update")
        # Mps, EnvLeft, EnvRight = InitMPS(Ns, Dp, Ds, Mpo)
        # Mps = OptMps(Mps, Mpo, EnvLeft, EnvRight, Ds, Method = 0)
        Mps, EnvLeft, EnvRight = InitMPS(Ns, Dp, Ds, Mpo)
        Mps, Eng1 = OptMps(Mps, Mpo, EnvLeft, EnvRight, Ds, Method)
        print("Energy per site:", Eng1/float(Ns))
        print("Energy averageL:", np.mean(Eng1/float(Ns)))
        sigmaX = [None] * Ns
        sigmaZ = [None] * Ns
        for site in range(Ns):
            sigmaX[site] = CalSigmaX_MPS(Mps, site)
            sigmaZ[site] = CalSigmaZ_MPS(Mps, site)
        print("Sigma_X per site:", sigmaX)
        print("Sigma_X average:", np.mean(sigmaX))
        print("Sigma_Z per site:", sigmaZ)
        print("Sigma_Z average:", np.mean(sigmaZ))

(Ns, Ds, g)=(10, 4, 0.428)
1-site Update
Energy per site: [-1.02813726 -1.02813726 -1.02813726 -1.02813726 -1.02813726 -1.02813726
 -1.02813726 -1.02813726 -1.02813726 -1.02813726]
Energy averageL: -1.0281372591359919
Sigma_X per site: [(0.9155284953906635-1.6791300938936748e-17j), (0.9066233915834957+1.3237037368513784e-17j), (0.9254967771019853-9.860917358684679e-17j), (0.9257237617926837-8.551457361788734e-17j), (0.9254678053889609-4.2960452293554045e-17j), (0.9254677222436142-2.700783163884119e-17j), (0.9257236332222627+1.1763964148382147e-18j), (0.9254967322422369-3.8721752394139645e-17j), (0.906623394535087-7.830209138073943e-17j), (0.9155285062112519-1.390511785525321e-17j)]
Sigma_X average: (0.9197680219712243-3.8739885992284643e-17j)
Sigma_Z per site: [(-1.4111132762284484e-08+1.5656498272118152e-18j), (-1.3309206570166765e-08+7.335011718842525e-17j), (-3.103839329998692e-08+3.790613667114705e-17j), (1.4835917883093686e-08-6.79957567555008e-17j), (-1.458614344307385e-07-4.1046

We note that $\langle Z_i\rangle$ is close to zero, we only have float error.

## Exact Diagonalization Method
Now we use exact diagonalization method for benchmark. We firstly define bit operation.

In [9]:
def ReadBit(i, n): # Read n-th bit
    return (i&(1<<n))>>n

def FlipBit(i, n): # Filp n-th bit 
    return i^(1<<n)

def PickBit(i, k, n): # Pick up nbits from k-th bit
    return (i&((2**n-1)<<k))>>k

def RotLBit(i, L, n): # circular bit shift left (for construcing translation operator)
    return (PickBit(i, 0, L-n)<<n) + (i>>(L-n))

Consider about the open boundary condition, we generate Cluster hopping list (3-body term) and Ising hopping list (2-body term).

In [10]:
def ClusterHopList(Ns):
    ClusterHopList = []
    for site in range(Ns-2):
        ClusterHopList.append([site, site+1, site+2])
    return ClusterHopList

def IsingHopList(Ns):
    IsingHopList = []
    for site in range(Ns-1):
        IsingHopList.append([site, site+1])
    return IsingHopList

Then we translate Cluster-Ising Hamiltonian into bit manipulation form. Then we store it to a sparse matrix.

In [11]:
from scipy import sparse
def ClusterIsingHam(g, Ns):
    """
    Args:
        g: Cluster interaction strength.
        Ns: Number of sites
    Returns:
        Hamr: Sparse Hamiltonian matrix 
    """
    HamFrom = []
    HamTo = []
    HamValue = []
    dim = 2** Ns
    ClusterList = ClusterHopList(Ns)
    IsingList = IsingHopList(Ns)
    for basis in range(dim):
        for ih in range(len(ClusterList)):
            Clu0 = ClusterList[ih][0]
            Clu1 = ClusterList[ih][1]
            Clu2 = ClusterList[ih][2]

            newbasis_one = FlipBit(basis, Clu0)
            HamTo.append(newbasis_one)
            HamFrom.append(basis)
            Val = g * (2 * ReadBit(basis, Clu1)-1) * (2 * ReadBit(basis, Clu2)-1)
            HamValue.append(Val)

            newbasis_two = FlipBit(basis, Clu2)
            HamTo.append(newbasis_two)
            HamFrom.append(basis)
            Val = g * (2 * ReadBit(basis, Clu0)-1) * (2 * ReadBit(basis, Clu1)-1)
            HamValue.append(Val)

        for ih in range(len(IsingList)):
            Clu0 = IsingList[ih][0]
            Clu1 = IsingList[ih][1]

            HamTo.append(basis)
            HamFrom.append(basis)
            Val = -1 * (2 * ReadBit(basis, Clu0)-1) * (2 * ReadBit(basis, Clu1)-1)
            HamValue.append(Val)

        for site in range(Ns):
            newbasis = FlipBit(basis, site)
            HamTo.append(newbasis)
            HamFrom.append(basis)
            HamValue.append(-1)
            
    Hamr = sparse.coo_matrix((HamValue,(HamTo, HamFrom)), shape=(dim, dim)).toarray()

    return Hamr

After defining sparse matrix of Hamiltonian. We then define functions of compute $\langle X_i \rangle$ and $\langle Z_i \rangle$ using bit operation.

In [12]:
def CalSigmaZ_ED(state, Ns):
    """
    Use Exact diagonlization method to compute Z_i.
    """
    probs = np.abs(state)**2
    dim = 2 ** Ns
    mz = [None] * Ns
    dim_array = np.arange(dim)
    for site in range(Ns):
        sigmaz = 2 * ReadBit(dim_array, site) - 1
        mz[site] = np.dot(probs, sigmaz)

        if np.abs(mz[site]) < 1.0e-12:
            mz[site] = 0
    return mz

def CalSigmaX_ED(state, Ns):
    """
    Use Exact diagonlization method to compute X_i.
    """
    mx = [None] * Ns
    dim = 2 ** Ns
    dim_array = np.arange(dim)
    for site in range(Ns):
        flip_index = FlipBit(dim_array, site)
        mx[site] = np.vdot(state, state[flip_index]).real


        if mx[site] < 1.0e-12:
            mx[site] = 0
    return mx

In [13]:
g = 0.428; Ns = 10
Hamr = ClusterIsingHam(g, Ns)
Eigvals, Eigvecs = LAs.eigsh(Hamr,k=1,which = 'SA')
E0 = float(Eigvals[0])
psi0 = Eigvecs[:, 0]
print("Exact Diagonalization method.")
print("Energy per site:", E0/float(Ns))
mx = CalSigmaX_ED(psi0, Ns)
print("Sigma_X per site:", mx)
print("Sigma_X average:", np.mean(mx))
mz = CalSigmaZ_ED(psi0, Ns)
print("Sigma_Z per site:", mz)
print("Sigma_Z average:", np.mean(mz))

Exact Diagonalization method.
Energy per site: -1.0281768119367765
Sigma_X per site: [np.float64(0.9151649740707714), np.float64(0.9062290337194011), np.float64(0.9247849470641332), np.float64(0.9248863912281435), np.float64(0.9247594360835767), np.float64(0.9247594360835762), np.float64(0.9248863912281431), np.float64(0.9247849470641327), np.float64(0.906229033719401), np.float64(0.915164974070771)]
Sigma_X average: 0.919164956433205
Sigma_Z per site: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Sigma_Z average: 0.0
